In [154]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [155]:
# ACADEMY WINNING MOVIES ON IMDB SORTED DESC BOX OFFICE HTML
imdb_html = requests.get('https://www.imdb.com/search/title/?release_date=2010-01-01,&sort=boxoffice_gross_us,desc&count=250')
soup = BeautifulSoup(imdb_html.content, 'html.parser')

In [156]:
# Scraped through the html code to find the div with the movies
main_div_tag = soup.find('div', {'class':'lister-list'})

In [157]:
# returns all 250 movies per page
movies_per_page = main_div_tag.find_all('div', {'class':'lister-item mode-advanced'})

In [158]:
# returns list of contents from page (250)
contents = []
for item in movies_per_page:
    contents.append(item.find('div',{'class':'lister-item-content'}))

In [159]:
# looks through contents and retrieves movie id (imdb)
movie_ids =[]
for i in contents:
    movie_ids.append(i.find('h3').find('a')['href'].split('/')[2])

In [160]:
# retrieves movie titles
movie_titles =[]
for i in contents:
    movie_titles.append(str(i.find('h3').find('a')).split('>')[1].split('<')[0])

In [161]:
# to retrieve movie release year
years =[]
for i in contents:
    year = str(i.find('h3').find('span', {'class':'lister-item-year text-muted unbold'})).split('(')[-1].split(')')[0]
    years.append(int(year))

In [162]:
# retrieves rating
ratings =[]
for i in contents:
    ratings.append(str(i.find('p').find('span',{'class':'certificate'})).split('>')[1].split('<')[0])

In [163]:
# retrieves runtime
runtimes =[]
for i in contents:
    runtimes.append(str(i.find('p').find('span',{'class':'runtime'})).split('>')[1].split('<')[0].split()[0])

In [164]:
# retrieves genres and cleans it, creates list of genres for each movie
genres = []
for i in contents:
    clean_genre = []
    genres_unclean = str(i.find('p').find('span',{'class':'genre'})).split('>')[1].split('<')[0].split('\n')[1].split(',')
    for genr in genres_unclean:
        clean_genre.append(genr.strip())
    genres.append(clean_genre)   

In [173]:
# Finds imdb rating
imdb_rating = []
for i in contents:
    imdb_rating.append(float(i.find('div',{'class':'inline-block ratings-imdb-rating'})['data-value']))

In [192]:
# Finds metascore rating
meta_score_rating = []
for i in contents:
    score = str(i.find('div',{'class':'inline-block ratings-metascore'}).find('span')).split('>')[1].split('<')[0].strip()
    meta_score_rating.append(int(score))

In [199]:
# for loop through each row in the table to retrieve the movie index number for imdb
imdb_popular_movies_indices = [row.a['href'].split('/')[2] for row in table_body_tag.find_all('tr')]

In [200]:
# API CALLS RETRIEVES list of 100 json files each representing one movie
pop_100_jsons = [requests.get(f"http://www.omdbapi.com/?i={i}&apikey=5a111996").json() for i in imdb_popular_movies_indices]

In [202]:
# Creating a pandas DataFrame and converting into CSV file
pd.DataFrame(pop_100_jsons).to_csv('imdb_100_popular_movies.csv')